<a href="https://colab.research.google.com/github/chiyeon01/CNN_Model_Mechanism/blob/main/pytorch/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install torchinfo

In [28]:
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 41.3 MB/s eta 0:00:00


In [29]:
# Module import
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchinfo
from torch.utils.data import Dataset, DataLoader
from torchvision import models
from torch.optim import SGD, Adagrad, RMSprop, Adam, AdamW
import torchmetrics

import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm

# Test Module
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchvision import transforms

In [71]:
class Config:
    batch_size = 8
    FineTune = False
    shuffle = False
    first_learning_rate = 1e-3
    second_learning_rate = 1e-4
    image_size = [224, 224]
    model_name = 'efficientnet_b0'
    make_summary = False
    callbacks = []
    metric = torchmetrics.classification.Accuracy(task="multiclass", num_classes=10)

In [67]:
# 사전학습된 모델을 만드는 함수(create pretrained model function)
# 모든 가중치는 DEFAULT로 선언(all weights = 'DEFAULT')
def create_pretrained_model(model_name, make_summary=False):
    if model_name == 'alexnet':
        model = models.alexnet(weights='DEFAULT')
    elif model_name == 'resnet50':
        model = models.resnet50(weights='DEFAULT')
    elif model_name == 'resnet101':
        model = models.resnet101(weights='DEFAULT')
    elif model_name == 'efficientnet_b0':
        model = models.efficientnet_b0(weights='DEFAULT')
        model.classifier = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(in_features=1280, out_features=10)
        )
    elif model_name == 'efficientnet_b4':
        model = models.efficientnet_b4(weights='DEFAULT')
    elif model_name == 'efficientnet_v2_s':
        model = models.efficientnet_v2_s(weighs='DEFAULT')
    elif model_name == 'efficientnet_v2_m':
        model = models.efficientnet_v2_m(weighs='DEFAULT')
    elif model_name == 'efficientnet_v2_l':
        model = models.efficientnet_v2_l(weighs='DEFAULT')
    elif model_name == 'convnext_base':
        model = models.convnext_base(weights='DEFAULT')
    elif model_name == 'convnext_small':
        model = models.convnext_small(weights='DEFAULT')
    else:
        print("🧨🧨 [ERROR] 모델 이름이 존재하지 않습니다. 🧨🧨")
        return None

    if make_summary:
        # 모델 정보 요약(model summary)
        print(torchinfo.summary(model, input_size=[1, 3] + Config.image_size,
                  col_names=['output_size', 'num_params', 'trainable'],
                  row_settings=['depth', 'var_names'],
                  depth=3))

    return model

In [68]:
model

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [88]:
# 1번 에포크 학습 함수(1 epoch train function)
def train_epoch(model=None, dataloader=None, loss_fn=None, metric=False, optimizer=None):
    if dataloader == None:
        print("🧨🧨 [ERROR] 데이터로더가 존재하지 않습니다. 🧨🧨")
        return None

    device = torch.device('cuda:0' if torch.cuda.is_available else 'cpu')

    model = model.to(device)
    metric = metric.to(device)

    # 평균 loss를 구하기 위한 변수(variable for average loss)
    avg_loss = 0
    sum_loss = 0

    # 모델 학습 설정(model train setting)
    model.train()

    with tqdm(total = len(dataloader), desc="[Training...] ", leave=True) as progress_bar:
        for batch_idx, (images, labels) in enumerate(dataloader):
            images = images.to(device)
            labels = labels.to(device)

            logits = model(images)
            loss = loss_fn(logits, labels)
            sum_loss += loss
            avg_loss = sum_loss / batch_idx

            acc = metric(F.softmax(logits, dim=-1).argmax(dim=-1), labels)

            # 옵티마이저 초기화(optimizer initialize)
            optimizer.zero_grad()

            # 오차역전파 계산(backpropagation)
            loss.backward()

            # 학습 파라미터 업데이트(parameter Update)
            optimizer.step()

            # Progress bar Update
            progress_bar.update(1)

            if batch_idx % 20 == 0:
                # Progress_bar Update -> set_postfix
                progress_bar.set_postfix({
                    "Loss" : avg_loss.item(),
                    "Accuracy" : metric.compute().item()
                    })

        metric.reset()

In [89]:
# 1번 에포크 학습 함수(1 epoch train function)
def validate_epoch(model=None, dataloader=None, loss_fn=None, metric=False):
    if dataloader == None:
        print("🧨🧨 [ERROR] 데이터로더가 존재하지 않습니다. 🧨🧨")
        return None

    device = torch.device('cuda:0' if torch.cuda.is_available else 'cpu')

    model = model.to(device)
    metric = metric.to(device)

    # 평균 loss를 구하기 위한 변수(variable for average loss)
    avg_loss = 0
    sum_loss = 0

    # 모델 평가로 설정(model evaluation setting)
    model.eval()

    with tqdm(total = len(dataloader), desc="[Validating..] ", leave=True) as progress_bar:
        for batch_idx, (images, labels) in enumerate(dataloader):
            images = images.to(device)
            labels = labels.to(device)

            logits = model(images)
            loss = loss_fn(logits, labels)
            sum_loss += loss
            avg_loss = sum_loss / batch_idx

            acc = metric(F.softmax(logits, dim=-1).argmax(dim=-1), labels)

            # Progress bar Update
            progress_bar.update(1)

            if batch_idx % 20 == 0:
                # Progress_bar Update -> set_postfix
                progress_bar.set_postfix({
                    "Validate_Loss" : avg_loss.item(),
                    "Validate_Accuracy" : metric.compute().item()
                    })

        metric.reset()

In [90]:
####### test #######

# transform 생성
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
])

# 데이터셋
train_data = datasets.FashionMNIST(root='data', train=True, download=True, transform=transform)
test_data = datasets.FashionMNIST(root='data', train=False, download=True, transform=transform)

train_dataloader = DataLoader(train_data, batch_size=Config.batch_size, shuffle=Config.shuffle, pin_memory=True)
test_dataloader = DataLoader(test_data, batch_size=Config.batch_size, shuffle=False, pin_memory=True)

model = create_pretrained_model(model_name=Config.model_name, make_summary=False)

loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=Config.first_learning_rate)

train_epoch(model, train_dataloader, loss_fn, Config.metric, optimizer)
validate_epoch(model, test_dataloader, loss_fn, Config.metric)

[Validating..] :  95%|█████████▍| 1184/1250 [00:19<00:01, 61.68it/s, Loss=0.799, Accuracy=0.762]


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 2.12 MiB is free. Process 4511 has 14.74 GiB memory in use. Of the allocated memory 14.58 GiB is allocated by PyTorch, and 19.46 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [64]:
model

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat